In [ ]:
#| default_exp string_templating

In [ ]:
#| exporti 

from archetypon.base_model import BaseModel
from pydantic import root_validator,validator
from typing import *
import string
import re

In [ ]:
#| export

def get_formatters_from_string(input_string:str)->List[str]:
    if input_string:
        keys = [tup[1] for tup in string.Formatter().parse(input_string) if tup[1] is not None]
        if len(keys)>0:
            return keys
        else:
            return {}

In [ ]:
#| export 

def string_to_dict(string, pattern):
    regex = re.sub(r'{(.+?)}', r'(?P<_\1>.+)', pattern)
    values = list(re.search(regex, string).groups())
    keys = re.findall(r'{(.+?)}', pattern)
    _dict = dict(zip(keys, values))
    return _dict

In [ ]:
my_string = 'I live in {city}, {state}, {country}.'
assert get_formatters_from_string(my_string) == ['city','state','country']

In [ ]:
#| export

class StringTemplate(BaseModel):
    string: Optional[str]=None
    template: str
    
    @classmethod
    def parse_string(cls,string):
        string_format = cls.__fields__['template'].default
        values = string_to_dict(string,string_format)
        return cls(**values)
    
    @validator('template',always=True)
    def validate_template(cls,v):
        template_fields = get_formatters_from_string(v)
        fields = [x for x in cls.__fields__.keys() if x not in ('template','string')]
        assert template_fields == fields,(template_fields,fields)
        return v
    
    @root_validator(skip_on_failure=True)
    def format_template(cls,values):

        values['string'] = values['template'].format(**values)

        return values
    
    def __init__(
        self,
        string=None, # positional only
        /,
        **kwargs
    ):
        if string: 
            obj = self.parse_string(string)
            super().__init__(**obj.dict())
        else:
            super().__init__(**kwargs)

In [ ]:
class IntroduceMe(StringTemplate):
    template:str = "Hi! My name is {first} {last}"
    first: str
    last: str

In [ ]:
IntroduceMe(first='Charlie',last='Schlinkert')

string,Hi! My name is Charlie Schlinkert
template,Hi! My name is {first} {last}
first,Charlie
last,Schlinkert


In [ ]:
IntroduceMe.parse_string("Hi! My name is Humphry Bogart")

string,Hi! My name is Humphry Bogart
template,Hi! My name is {first} {last}
first,Humphry
last,Bogart


In [ ]:
IntroduceMe("Hi! My name is James Bond")

string,Hi! My name is James Bond
template,Hi! My name is {first} {last}
first,James
last,Bond
